Course Instructor: Bernd Neumayr, JKU

# UE09: Beispiel zu Bayesian Networks

Zwei Mitarbeiter:innen (Johanna und Peter) kommen manchmal zu spät zur Arbeit. Bei Peter ist eine bekannte  Ursache des Zuspätkommens der Ausfall seines Weckers. Ein Bahnstreik kann sowohl bei Peter als auch bei Johanna ein Zuspätkommen verursachen. Erstellen Sie die Struktur des Bayes’schen Netzes mit den Zufallsvariablen JohannaLate (J), PeterLate (M), TrainStrike (T), PetersAlarmFails (A). Dies sind boolesche Zufallsvariablen, sie haben also nur True und False als mögliche Ausprägungen. Folgende (bedingte) Wahrscheinlichkeiten sind bekannt:

* Die Wahrscheinlichkeit, dass Peter’s Wecker ausfällt liegt bei 10 %.
    * P(A=true) = 0,1
* Die Wahrscheinlichkeit eines Bahnstreiks liegt bei 5 %.
    * P(T=true) = 0,05
* Die Wahrscheinlichkeit dass Johanna zu spät in die Arbeit kommt liegt im Falle eines Bahnstreiks bei 50 % und sonst bei 2 %.
    * P(J=true | T=true) = 0,5
    * P(J=true | T=false) = 0,02
* Die Wahrscheinlichkeit, dass Peter zu spät in die Arbeit kommt liegt im Falle eines Bahnstreiks bei 70 % bei Weckerausfall und 30 % bei funktionierendem Wecker; wenn kein Bahnstreik ist, dann liegt die Wahrscheinlichkeit seines Zuspätkommens bei 40 % bei Weckerausfall und bei 1 % wenn der Wecker funktioniert.
    * P(P=true | T=true, A=true) = 0,7
    * P(P=true | T=true, A=false) = 0,3
    * P(P=true | T=false, A=true) = 0,4
    * P(P=true | T=false, A=false) = 0,01

Ihre Aufgaben
2. Umsetzung und Beantwortung von Fragen mittels pgmpy. Abzugeben ist ein Jupyter Notebook. Sie können sich bei Ihrer Lösung an https://github.com/jku-win-dke/SemAI/blob/main/V08_PGM.ipynb orientieren. (insgesamt 5 Punkte) Um die Conditional Probability Tables (CPDs) zwischen den beiden Umsetzungsvarianten einfach vergleichbar zu machen, verwenden Sie bitte in beide Umsetzungsvarianten die selben Variablen- und Zustandsnamen. Hier der entsprechende Code für pgmpy:

statenames = {
'TrainStrike': ['True', 'False'],
'PeterLate': ['True', 'False'],
'JohannaLate': ['True', 'False'],
'PetersAlarmFails': ['True', 'False']}

Es sind jeweils folgende Aufgaben zu erfüllen bzw Fragen zu beantworten (jeweils 1 Punkt):

a) Erstellung des Bayes’schen Netzes (Struktur + Wahrscheinlichkeiten)

b) Wie hoch ist die Wahrscheinlichkeit, dass Johanna bzw Peter zu spät in die Arbeit kommen?

c) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, falls Peters Wecker nicht funktioniert?

d) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt?

e) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt obwohl Peters Wecker funktioniert


##Preparations

In [ ]:
!pip install -q pgmpy networkx matplotlib plotly daft

## a) Erstellung des Bayes’schen Netzes (Struktur + Wahrscheinlichkeiten)

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

statenames = {
    'TrainStrike': ['True', 'False'],
    'PeterLate': ['True', 'False'],
    'JohannaLate': ['True', 'False'],
    'PetersAlarmFails': ['True', 'False']
}

model = BayesianModel([
    ('TrainStrike', 'JohannaLate'),
    ('TrainStrike', 'PeterLate'),
    ('PetersAlarmFails', 'PeterLate')
])

# Wahrscheinlichkeit für PetersAlarmFails
cpd_peters_alarm_fails = TabularCPD(
    variable='PetersAlarmFails',
    variable_card=2,
    values=[[0.1], [0.9]],
    state_names=statenames
)

# Wahrscheinlichkeit für TrainStrike
cpd_train_strike = TabularCPD (
    variable='TrainStrike',
    variable_card=2,
    values=[[0.05], [0.95]],
    state_names=statenames
)

# Wahrscheinlichkeit für JohannaLate
cpd_johanna_late = TabularCPD(
    variable='JohannaLate',
    variable_card=2,
    values=[[0.5, 0.02],
            [0.5, 0.98]],
    evidence=['TrainStrike'],
    evidence_card=[2],
    state_names=statenames
)

# Wahrscheinlichkeit für PeterLate
cpd_peter_late = TabularCPD (
    variable='PeterLate',
    variable_card=2,
    values=[[0.7, 0.3, 0.4, 0.01],
            [0.3, 0.7, 0.6, 0.99]],
    evidence=['TrainStrike', 'PetersAlarmFails'],
    evidence_card=[2, 2],
    state_names=statenames
)

model.add_cpds(cpd_train_strike, cpd_peter_late, cpd_johanna_late, cpd_peters_alarm_fails)
model.check_model()

for cpd in model.get_cpds():
    print(cpd)

## b) Wie hoch ist die Wahrscheinlichkeit, dass Johanna bzw Peter zu spät in die Arbeit kommen?

In [ ]:
inference = VariableElimination(model)

johanna_late_prob = inference.query(variables=['JohannaLate'])
peter_late_prob = inference.query(variables=['PeterLate'])
print("Probability of Johanna being late:")
print(johanna_late_prob)
print("Probability of Peter being late:")
print(peter_late_prob)

## c) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, falls Peters Wecker nicht funktioniert?

In [ ]:
evidence = {'PetersAlarmFails': 'True'}
print(inference.query(variables=['JohannaLate'], evidence=evidence))

## d) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt?

In [ ]:
print(inference.query(variables=['JohannaLate'], evidence={'PeterLate': 'True'}))

## e) Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät in die Arbeit kommt obwohl Peters Wecker funktioniert

In [ ]:
print(inference.query(variables=['JohannaLate'], evidence={'PeterLate': 'True', 'PetersAlarmFails': 'False'}))